In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
dataset = pd.read_csv('/content/drive/My Drive/data/fer2013.csv')

In [0]:
x_train, y_train, x_test, y_test = [], [], [], []

for index, row in dataset.iterrows():
    temp = row['pixels'].split(' ')
    try:
        if 'Training' in row['Usage']:
            x_train.append(np.array(temp, 'float32'))
            y_train.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
            x_test.append(np.array(temp, 'float32'))
            y_test.append(row['emotion'])
    except:
        print("Error occured at", index, "index and", row, "row")


In [6]:
from keras import utils as np_utils
x_train = np.array(x_train, 'float32')
x_test = np.array(x_test, 'float32')
y_train = np.array(y_train, 'float32')
y_test = np.array(y_test, 'float32')

y_train = np_utils.to_categorical(y_train, num_classes=7)
y_test = np_utils.to_categorical(y_test, num_classes=7)

Using TensorFlow backend.


In [0]:
x_train /= 255.0
x_test /= 255.0

In [0]:
x_train = x_train.reshape(-1, 48, 48, 1)
x_test = x_test.reshape(-1, 48, 48, 1)

In [0]:
model = keras.Sequential([
    keras.layers.Conv2D(64, (3, 3), activation='relu',
                        input_shape=(48, 48, 1), kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Dropout(0.5),
    keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Dropout(0.5),
    keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Dropout(0.5),
    keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Dropout(0.5),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(7, activation='softmax')
])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 23, 23, 128)       5

In [0]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
model.fit(
    x_train,
    y_train,
    epochs=100,
    batch_size=64,
    validation_split=0.2,
    shuffle=True,
)

Epoch 1/100
359/359 [==============================] - 53s 147ms/step - loss: 2.1350 - accuracy: 0.1988 - val_loss: 1.8779 - val_accuracy: 0.2517
Epoch 2/100
359/359 [==============================] - 51s 141ms/step - loss: 1.8692 - accuracy: 0.2302 - val_loss: 1.8394 - val_accuracy: 0.2517
Epoch 3/100
359/359 [==============================] - 51s 141ms/step - loss: 1.8441 - accuracy: 0.2464 - val_loss: 1.8323 - val_accuracy: 0.2517
Epoch 4/100
359/359 [==============================] - 51s 141ms/step - loss: 1.8354 - accuracy: 0.2491 - val_loss: 1.8236 - val_accuracy: 0.2517
Epoch 5/100
359/359 [==============================] - 51s 141ms/step - loss: 1.8295 - accuracy: 0.2505 - val_loss: 1.8254 - val_accuracy: 0.2517
Epoch 6/100
359/359 [==============================] - 51s 141ms/step - loss: 1.8251 - accuracy: 0.2500 - val_loss: 1.8137 - val_accuracy: 0.2517
Epoch 7/100
359/359 [==============================] - 51s 141ms/step - loss: 1.8185 - accuracy: 0.2507 - val_loss: 1.8076 -

In [0]:
fer_json = model.to_json()

In [0]:
with open("drive/My Drive/fer.json", "w") as json_file:
    json_file.write(fer_json)

In [0]:
model.save_weights("drive/My Drive/fer.h5")